In [2]:
from bs4 import BeautifulSoup
import requests
import pandas
# from pandas.tools.merge import concat
from pandas.io import sql
# import numpy as np
from unidecode import unidecode
import time
import random
import re
import MySQLdb
import sqlalchemy

In [3]:
# MySQL Variables
the_host = "127.0.0.1"
the_db = "mikes_db"
the_user = "root"
the_pw = "Prog2016"
db = MySQLdb.connect(the_host, the_user, the_pw, the_db, use_unicode = True, charset = "utf8")
the_cursor = db.cursor()

In [4]:
# Add distinctive User-Agent header to let the server know who you are
headers = {
    'User-Agent': 'MikesBballAnalysisScript/Contact me: mrhodes262@gmail.com',
    'Accept-Encoding': 'gzip, deflate',
}

# Base URLs
espn_ncaa_bball_team_url = 'http://espn.go.com/mens-college-basketball/teams'
team_schedule_base_url = 'http://espn.go.com/mens-college-basketball/team/schedule/_/id/'
base_roster_url = 'http://espn.go.com/mens-college-basketball/team/roster/_/id/'
box_score_base_url = 'http://espn.go.com/ncb/boxscore?gameId='
play_by_play_base_url = 'http://espn.go.com/ncb/playbyplay?gameId='

In [5]:
def get_team_list():
    # Create an empty list that will contain school names and URLs to their ESPN profiles
    team_link_list = []
    
    # Request the HTML doc from ESPN's website
    bball_team_request = requests.get(espn_ncaa_bball_team_url, headers = headers)
    # Retrieve the contents and create beautiful soup object
    bball_team_request_content = BeautifulSoup(bball_team_request.content)
     
    # The school names are contained in unordered list tags named "medium-logos". Find each ul tag with that class name.
    blah = bball_team_request_content.findAll('ul', {'class': 'medium-logos'})
     
    # Each one of those ul tags represents a conference
    for unorderedlist in blah:
        # Within those conferences, find li tags, which will contain the names and links for each school in the conference
        conference = unorderedlist.findAll('li')
         
        # For each li that represents the school, get the schools name and ESPN profile link
        for school in conference:
            school_name = school.a.contents[0]
            school_name = unidecode(school_name)
            school_link = school.a.get('href')
            school_link = unidecode(school_link)
            school_id = school_link.split('/')
             
            # Create a list containing the school's info
            school_link = [school_name, school_link, school_id[7]]
            # Add that school's list to the master list
            team_link_list.append(school_link)
            
    team_link_list_df = pandas.DataFrame(team_link_list, columns = ['school_name', 'espn_profile_link', 'espn_school_id'])
#     print team_link_list_df
#     sql.to_sql(team_link_list_df, con=db, name='ncaa_team_lists_2015', if_exists='append', flavor='mysql', index =False)
    return team_link_list

In [6]:
team_link_list = get_team_list()

/home/mrhodes/.anaconda2/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
/home/mrhodes/.anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:23: RuntimeWarning: Argument <type 'str'> is not an unicode object. Passing an encoded string will likely have unexpected results.


In [7]:
def get_team_schedule_results(team_link_list, write_to_sql=False):
    team_link_list = team_link_list
    all_schools_parsed_season_results = []

    pandas_df_column_headers = ['game_date', 'game_id', 'school_id', 'opponent_school_id', 'opponent_school_link', 'home_away_neutral', 'game_outcome', 
                            'team_score', 'opponent_score', 'overtime', 'team_overall_cuml_wins', 'team_overall_cuml_losses', 
                            'team_conf_cuml_wins', 'team_conf_cuml_losses']
    
    # For each school in "team_link_list"
    for school in team_link_list:
        
        # Empty list to put rows with HTML
        team_season_results = []
    
        team_schedule_url = team_schedule_base_url + str(school[2])
        the_team_schedule_request = requests.get(team_schedule_url, headers = headers)
        the_team_schedule = BeautifulSoup(the_team_schedule_request.content)
        bb = the_team_schedule.findAll('table', {'class': 'tablehead'})
    
        for table in bb:
            rows = table.findAll('tr', {'class': re.compile("team-41")})
            
            # The first two rows are the team name and column headers respectively. We dont want to parse those.
            for idx, row in enumerate(rows):
    
                team_season_results.append(row.findAll('td'))
            
            # Each element in the list now represents a game. Each game is a list with game info.
            for game in team_season_results:
                
                # If the game has been played and hasnt been postponed, continue
                if (str(game[2].find('li', {'class': 'score'})) != None and
                    game[2].find('li', {'class': re.compile('game-status')}) != None):
                    
                    game_date = game[0].contents[0]
                    try:
                        opponent_school_link  = game[1].a.get('href')
                        opponent_school_link = unidecode(opponent_school_link)
                        opponent_school_id = opponent_school_link.split('/')[7]
                    except AttributeError:
                        opponent_school_link  = None
                        opponent_school_id = None
                    
                    
                    # This yields a list of html elements about the team. We are interested in if there is a * (star), which indicates if the game was played at a
                    # neutral location. It will be the last element in the school list (if it exists).
                    opponent_name = game[1].find('li', {'class': 'team-name'}).contents
                    # if the list is > 1, it has some extra info about the school/game location (e.g. ranking, neutral location)
                    if len(opponent_name) > 1:
                        # Neutral location will always be at the end if it exists. Check for it and if it was at a neutral location, make "neutral_location" variable = True
                        if opponent_name[len(opponent_name)- 1] == '*':
                            neutral_location = True
                        else:
                            neutral_location = False
                    else:
                        neutral_location = False
                    
                    # If neutral_location == True, then give "home_away_neutral" vble a value of neutral
                    if neutral_location:
                        home_away_neutral = 'neutral'
                    # otherwise, look for the vs or @ symbol from the table and assign accordingly
                    else:
                        if game[1].find('li', {'class': 'game-status'}).contents[0] == 'vs':
                            home_away_neutral = 'home'
                        else:
                            home_away_neutral = 'away'
                    
                    
                    # Was it a Win or a Loss?
                    game_outcome = game[2].find('li', {'class': re.compile('game-status')}).span.contents[0]
                    # Game score will have score AND Overtime info (if relevant). We split on ' '. If the resulting list > 0, then there was an overtime.
                    game_outcome_detail = game[2].find('li', {'class': 'score'}).a.contents[0].split(' ')
                
                    # Was there overtime played?
                    if len(game_outcome_detail) > 1:
                        overtime = True
                    else:
                        overtime = False
                    
                    game_score = game_outcome_detail[0]
                    game_score_split = game_score.split("-")
                    
                    # ESPN always list the winning number of points first, the loosing second. If the team we are lookign at won, give them the first number of points.
                    if game_outcome == "W":
                        team_score = game_score_split[0]
                        opponent_score = game_score_split[1]
                    else:
                        team_score = game_score_split[1]
                        opponent_score = game_score_split[0]
                        
                    # Get the gameid so that we can use it to get play-by-play information about the game.
                    game_id = game[2].find('li', {'class': 'score'}).a.get('href').split("=")[1]
                    
                    if game[3].contents[0] != '--':
                        team_record = game[3].contents[0].replace('(', '')
                        team_record = team_record.replace(')', '').split(' ')
                        team_overall_cuml_wins = team_record[0].split('-')[0]
                        team_overall_cuml_losses = team_record[0].split('-')[1]
                        team_conf_cuml_wins = team_record[1].split('-')[0]
                        team_conf_cuml_losses = team_record[1].split('-')[1]
                    else:
                        team_record = None
                        team_record = None
                        team_overall_cuml_wins = None
                        team_overall_cuml_losses = None
                        team_conf_cuml_wins = None
                        team_conf_cuml_losses = None
        
                    game_results = [game_date, game_id, school[2], opponent_school_id, opponent_school_link, home_away_neutral, game_outcome, team_score, opponent_score, 
                                           overtime, team_overall_cuml_wins, team_overall_cuml_losses, team_conf_cuml_wins, team_conf_cuml_losses]
                    all_schools_parsed_season_results.append(game_results)
            print str(school[2])
                    
        team_parsed_season_results_pandas = pandas.DataFrame(all_schools_parsed_season_results, columns = pandas_df_column_headers)
#         team_parsed_season_results_pandas.to_csv('/home/mrhodes/Documents/Code/Eclipse_Workspaces/NCAABasketballAnalysis/AllSchools_Season_Results2.csv')
        # sprcify if you want to write to sql or not
        if write_to_sql == True:
            sql.to_sql(team_parsed_season_results_pandas, con=db, name='ncaa_team_results_2015', if_exists='append', flavor='mysql')
        time.sleep(3)
    return all_schools_parsed_season_results

In [ ]:
get_team_schedule_results(team_link_list)

399
2066
42
311
160
2349
2378
261
2132
41
151
248
235
2567
58
218
2655
202
2116
2166
2168
2184
2230
2244
45
2325
113
227
257
2603
139
179